In [1]:
from pprint import pprint
from convertmol import parse_sdf_file, bond_type_dict, single_bond_stereo_dict, double_bond_stereo_dict
from torch_geometric.data import Data
from torch_geometric.nn import *
from torch_geometric.utils import to_dense_adj, to_dense_batch, add_self_loops
from torch_geometric.nn.conv import MessagePassing
import torch
from torch import nn
import rdkit
from tqdm import tqdm
import itertools
from rdkit import Chem
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt
from rdkit import RDLogger
from copy import deepcopy
from torch.utils.data import Dataset, DataLoader
# Suppress RDKit warnings
RDLogger.DisableLog('rdApp.*')
import sascorer
torch.set_default_device('cpu:0')#'cuda'

In [2]:
import threading
from time import sleep, time
sem = threading.Semaphore(2)

In [4]:
AID=331168

In [71]:
target_info_url=f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{AID}/targets/ProteinGI,ProteinName,GeneID,GeneSymbol/JSON"
download_url=f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{AID}/JSON"

In [61]:
f=open("temp","rb")

In [63]:
os.path.getsize("temp")

321

In [66]:
def wget(url,fname='temp',download=False,output=True):
    import requests
    from tqdm import tqdm
    
    url = url
    with sem:
        sleep(1)
        r = requests.get(url, stream=True)
        
        if download and (r.status_code//100)==2:
            with open(fname, 'wb') as f:
                if r.headers.get('content-length'):
                    total_length = int(r.headers.get('content-length'))
                    for chunk in tqdm(r.iter_content(chunk_size=1024), expected_size=(total_length/1024) + 1): 
                        if chunk:
                            f.write(chunk)
                            f.flush()
                else:
                    for chunk in r.iter_content(chunk_size=1024): 
                        if chunk:
                            f.write(chunk)
                            f.flush()
            if output:
                with open(fname, 'rb') as f:
                    return f.read()
        else:
            r = requests.get(url)
            if download:
                with open(fname, 'wb') as f:
                    f.write(r.content)
            if output:
                return r.content

In [67]:
def download_target_info():
    import json
    return json.loads(wget(target_info_url).decode())

In [77]:
def download_data():
    import json
    return json.loads(wget(download_url).decode())

In [82]:

# should assume JSON_RESP['PC_AssaySubmit']['assay']['descr']['activity_outcome_method']==2
OUTCOME_MAPPING = {
    1: "Inactive",
    2: "Active",
    3: "Inconclusive",
    4: "Unspecified",
    5: "Probe"
}

In [73]:
res=download_target_info()

In [74]:
print('targets:',', '.join(res['InformationList']['Information'][0]['GeneSymbol']))

targets: ACE2


In [75]:
res2=download_data()

In [83]:
for i in res2['PC_AssaySubmit']['data']:
    print(i['sid'],OUTCOME_MAPPING[i['outcome']])

103560567 Active
103565682 Unspecified
103565727 Unspecified
103565729 Unspecified
103565772 Unspecified
103565911 Active
103565951 Unspecified
103565955 Unspecified
103565990 Active
103565994 Active
103565996 Active
103566036 Active
103566087 Active
103566089 Active
103566128 Active
103566131 Active
103566131 Active
103566132 Active
103566172 Active
103566173 Active
103566206 Active
103566207 Active
103566210 Active
103566278 Active
103566278 Active
103566321 Active


In [76]:
res2

{'PC_AssaySubmit': {'assay': {'descr': {'aid': {'id': 331168, 'version': 8},
    'aid_source': {'db': {'name': 'ChEMBL',
      'source_id': {'str': 'CHEMBL929264'}}},
    'name': 'Inhibition of ACE2',
    'description': ['Title: Development of potent and selective phosphinic peptide inhibitors of angiotensin-converting enzyme 2.',
     '',
     'Abstract: Angiotensin-converting enzyme 2 (ACE2), a recently identified human homologue of angiotensin-converting enzyme, is a zinc metallocarboxypeptidase which may play a unique role in cardiovascular and renal function. Here we report the discovery of potent and selective inhibitors of ACE2, which have been identified by evaluating a series of phosphinic di- and tripeptides of the general formula: Z-Xaa(PO 2-CH 2)YaaOH and Ac-Zaa-Xaa(PO 2-CH 2)YaaOH. The most potent inhibitor in this series is a tripeptide that displays a K i value of 0.4 nM toward ACE2 and is 3 orders of magnitude less potent toward carboxypeptidase A. Phosphinic tripeptide